In [17]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy

from datetime import datetime
import statsmodels.formula.api as smf


from matplotlib import style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from matplotlib.font_manager import FontProperties
from pylab import mpl

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_columns', None)

In [18]:
cross = pd.read_csv('/Users/liuwanting/Desktop/pythonhomework/TRD_Mnth202509.csv')
from pandas.tseries.offsets import MonthEnd
cross['month'] = pd.to_datetime(cross['Trdmnt'], format='%Y-%m') + MonthEnd(1)
cross['Stkcd'] = cross['Stkcd'].apply(lambda x: '{:0>6}'.format(x))
cross.rename(columns={'Mretwd': 'Return', 'Msmvosd': 'floatingvalue', 'Msmvttl': 'totalvalue'}, inplace=True)

rf_data = pd.read_csv('/Users/liuwanting/Desktop/pythonhomework/Marketret_mon_stock2024.csv')
rf_data['month'] = pd.to_datetime(rf_data['month'], format='%b %Y') + MonthEnd(1)
rf_data = rf_data[['month', 'rfmonth']]

cross = pd.merge(cross, rf_data, on='month', how='left')

cross = cross.sort_values(by=['Stkcd', 'month'])
cross['list_month'] = cross.groupby('Stkcd').cumcount() + 1

cross['ret'] = cross['Return'] - cross['rfmonth']
cross['floatingvalue'] = cross['floatingvalue'] * 1000
cross['totalvalue'] = cross['totalvalue'] * 1000

cross

,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,floatingvalue,totalvalue,Ndaytrd,Return,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M,month,rfmonth,list_month,ret
0,000001,1991-04,3,49.00,30,43.68,13400,6.150000e+05,1.157520e+09,2.118487e+09,20,NaN,NaN,4,1991-04-03,NaN,NaN,1991-04-30,0.006651,1,NaN
1,000001,1991-05,2,43.46,31,38.34,187800,7.675000e+06,1.016010e+09,1.859497e+09,24,-0.122253,0.235714,4,1991-04-03,NaN,NaN,1991-05-31,0.006092,2,-0.128345
2,000001,1991-06,1,38.53,28,33.99,30600,1.094000e+06,9.007350e+08,1.648521e+09,23,-0.113459,-0.113459,4,1991-04-03,NaN,NaN,1991-06-30,0.006092,3,-0.119551
3,000001,1991-07,1,33.65,31,29.54,6100,1.940430e+05,7.828100e+08,1.432695e+09,16,-0.130921,-0.130921,4,1991-04-03,NaN,NaN,1991-07-31,0.006092,4,-0.137013
4,000001,1991-08,1,29.39,31,15.00,3243100,4.957624e+07,6.748338e+08,1.346275e+09,15,-0.411588,-0.411587,4,1991-08-01,NaN,NaN,1991-08-31,0.006092,5,-0.417680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886908,920992,2025-05,6,20.00,30,20.46,92609592,1.922463e+09,5.707967e+08,1.979115e+09,19,0.051312,0.051715,64,2023-04-25,0.0,0.0,2025-05-31,NaN,32,NaN
886909,920992,2025-06,3,20.40,30,21.26,73152031,1.580767e+09,5.931152e+08,2.056500e+09,20,0.039101,0.039101,64,2023-04-25,0.0,0.0,2025-06-30,NaN,33,NaN
886910,920992,2025-07,1,21.13,31,22.44,96064221,2.126994e+09,6.260351e+08,2.170642e+09,23,0.055503,0.055503,64,2023-04-25,0.0,0.0,2025-07-31,NaN,34,NaN
886911,920992,2025-08,1,22.12,29,21.65,63953451,1.432608e+09,6.039955e+08,2.094225e+09,21,-0.035205,-0.035205,64,2023-04-25,0.0,0.0,2025-08-31,NaN,35,NaN


In [19]:
all_months = pd.DataFrame(cross['month'].unique(), columns=['month'])
all_stocks = pd.DataFrame(cross['Stkcd'].unique(), columns=['Stkcd'])
full_index = all_stocks.merge(all_months, how='cross')

cross_full = full_index.merge(cross, on=['Stkcd', 'month'], how='left')
cross_full = cross_full.sort_values(['Stkcd', 'month'])
cross_full['next_ret'] = cross_full.groupby('Stkcd')['ret'].shift(-1)
cross = cross.merge(cross_full[['Stkcd', 'month', 'next_ret']],
                    on=['Stkcd', 'month'], how='right')
cross['Cumsum_tradingday'] = cross.groupby('Stkcd')['Ndaytrd'].transform(lambda x: x.rolling(window=12, min_periods=1).sum())
print("方法1完成：补全日期序列后使用shift")

cross = cross[(cross['month'] >= '1995-01-31') & (cross['month'] <= '2024-12-31')]
cross

方法1完成：补全日期序列后使用shift


,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,floatingvalue,totalvalue,Ndaytrd,Return,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M,month,rfmonth,list_month,ret,next_ret,Cumsum_tradingday
49,000001,1995-01,3.0,10.50,27.0,10.54,17488100.0,1.867690e+08,3.134297e+09,4.543463e+09,19.0,0.005725,0.005725,4.0,1994-09-02,NaN,NaN,1995-01-31,0.008719,46.0,-0.002994,-0.000180,247.0
50,000001,1995-02,13.0,10.60,28.0,10.63,8544200.0,9.192790e+07,3.161060e+09,4.582260e+09,12.0,0.008539,0.008539,4.0,1994-09-02,NaN,NaN,1995-02-28,0.008719,47.0,-0.000180,0.003511,244.0
51,000001,1995-03,1.0,10.63,31.0,10.76,18810600.0,2.059094e+08,3.199719e+09,4.638299e+09,23.0,0.012230,0.012230,4.0,1994-09-02,NaN,NaN,1995-03-31,0.008719,48.0,0.003511,-0.108161,244.0
52,000001,1995-04,3.0,10.78,28.0,9.69,13235500.0,1.380604e+08,2.881531e+09,4.177055e+09,20.0,-0.099442,-0.099442,4.0,1994-09-02,NaN,NaN,1995-04-30,0.008719,49.0,-0.108161,0.002633,243.0
53,000001,1995-05,2.0,9.69,31.0,9.80,53197900.0,6.049730e+08,2.914242e+09,4.224473e+09,22.0,0.011352,0.011352,4.0,1994-09-02,NaN,NaN,1995-05-31,0.008719,50.0,0.002633,-0.073005,244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453228,920992,2024-08,1.0,8.50,30.0,8.19,14436632.0,1.185912e+08,2.284861e+08,7.922264e+08,22.0,-0.037603,-0.037603,64.0,2023-04-25,0.0,0.0,2024-08-31,0.001118,23.0,-0.038721,0.276049,242.0
2453229,920992,2024-09,2.0,8.15,30.0,10.46,18109819.0,1.566140e+08,2.918149e+08,1.011806e+09,19.0,0.277167,0.277167,64.0,2023-04-25,0.0,0.0,2024-09-30,0.001118,24.0,0.276049,0.389895,241.0
2453230,920992,2024-10,8.0,13.02,31.0,14.55,86774237.0,1.145494e+09,4.059185e+08,1.407435e+09,18.0,0.391013,0.391013,64.0,2023-04-25,0.0,0.0,2024-10-31,0.001118,25.0,0.389895,0.241494,242.0
2453231,920992,2024-11,1.0,14.69,29.0,18.08,130783392.0,2.346534e+09,5.043990e+08,1.748895e+09,21.0,0.242612,0.242612,64.0,2023-04-25,0.0,0.0,2024-11-30,0.001118,26.0,0.241494,-0.243928,241.0


### 导入市场收益率数据

In [20]:
from pandas.tseries.offsets import MonthEnd
Market_ret = pd.read_csv('/Users/liuwanting/Desktop/pythonhomework/Marketret_mon_stock2024.csv')
Market_ret['month'] = pd.to_datetime(Market_ret['month'], format='%b %Y') + MonthEnd(0)
Market_ret.set_index('month', inplace=True)
Market_ret.sort_index(inplace=True)
Market_ret = Market_ret.drop(columns=['Unnamed: 0'])
Market_ret.rename(columns={'ret': 'MKT'}, inplace=True)
Market_ret

,MarketR,MarketR_e,rfmonth,MKT,ret_e,marketret3,marketret6,marketret12,Q
month,,,,,,,,,
1991-01-31,0.029998,0.036554,0.006930,0.023068,0.029624,-0.084127,-0.305662,0.254049,1991 Q1
1991-02-28,0.010203,0.021860,0.006930,0.003273,0.014930,-0.183573,-0.384745,0.241492,1991 Q1
1991-03-31,-0.099663,-0.060425,0.006930,-0.106593,-0.067355,-0.252928,-0.445049,0.288857,1991 Q1
1991-04-30,-0.079652,-0.031449,0.006651,-0.086303,-0.038100,-0.234776,-0.394937,0.691749,1991 Q2
1991-05-31,-0.074521,0.005375,0.006092,-0.080613,-0.000717,-0.236294,0.181673,1.542701,1991 Q2
...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.031468,-0.033685,0.001118,-0.032586,-0.034803,0.151050,NaN,NaN,2024 Q3
2024-09-30,0.195992,0.227078,0.001118,0.194874,0.225960,0.200147,NaN,NaN,2024 Q3
2024-10-31,-0.003409,0.070220,0.001118,-0.004527,0.069102,0.000228,NaN,NaN,2024 Q4


In [21]:
cross = pd.merge(cross,Market_ret[['MKT']],left_on='month',right_on='month',how='left')
cross

,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,floatingvalue,totalvalue,Ndaytrd,Return,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M,month,rfmonth,list_month,ret,next_ret,Cumsum_tradingday,MKT
0,000001,1995-01,3.0,10.50,27.0,10.54,17488100.0,1.867690e+08,3.134297e+09,4.543463e+09,19.0,0.005725,0.005725,4.0,1994-09-02,NaN,NaN,1995-01-31,0.008719,46.0,-0.002994,-0.000180,247.0,-0.125017
1,000001,1995-02,13.0,10.60,28.0,10.63,8544200.0,9.192790e+07,3.161060e+09,4.582260e+09,12.0,0.008539,0.008539,4.0,1994-09-02,NaN,NaN,1995-02-28,0.008719,47.0,-0.000180,0.003511,244.0,-0.022778
2,000001,1995-03,1.0,10.63,31.0,10.76,18810600.0,2.059094e+08,3.199719e+09,4.638299e+09,23.0,0.012230,0.012230,4.0,1994-09-02,NaN,NaN,1995-03-31,0.008719,48.0,0.003511,-0.108161,244.0,0.122163
3,000001,1995-04,3.0,10.78,28.0,9.69,13235500.0,1.380604e+08,2.881531e+09,4.177055e+09,20.0,-0.099442,-0.099442,4.0,1994-09-02,NaN,NaN,1995-04-30,0.008719,49.0,-0.108161,0.002633,243.0,-0.113023
4,000001,1995-05,2.0,9.69,31.0,9.80,53197900.0,6.049730e+08,2.914242e+09,4.224473e+09,22.0,0.011352,0.011352,4.0,1994-09-02,NaN,NaN,1995-05-31,0.008719,50.0,0.002633,-0.073005,244.0,0.166736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112835,920992,2024-08,1.0,8.50,30.0,8.19,14436632.0,1.185912e+08,2.284861e+08,7.922264e+08,22.0,-0.037603,-0.037603,64.0,2023-04-25,0.0,0.0,2024-08-31,0.001118,23.0,-0.038721,0.276049,242.0,-0.032586
2112836,920992,2024-09,2.0,8.15,30.0,10.46,18109819.0,1.566140e+08,2.918149e+08,1.011806e+09,19.0,0.277167,0.277167,64.0,2023-04-25,0.0,0.0,2024-09-30,0.001118,24.0,0.276049,0.389895,241.0,0.194874
2112837,920992,2024-10,8.0,13.02,31.0,14.55,86774237.0,1.145494e+09,4.059185e+08,1.407435e+09,18.0,0.391013,0.391013,64.0,2023-04-25,0.0,0.0,2024-10-31,0.001118,25.0,0.389895,0.241494,242.0,-0.004527
2112838,920992,2024-11,1.0,14.69,29.0,18.08,130783392.0,2.346534e+09,5.043990e+08,1.748895e+09,21.0,0.242612,0.242612,64.0,2023-04-25,0.0,0.0,2024-11-30,0.001118,26.0,0.241494,-0.243928,241.0,0.008606


### 估计个股的β

In [22]:
import statsmodels.api as sm

def rolling_beta_per_stock(data, window_months=60):
    min_periods = max(1, int(np.ceil(window_months * 2 / 3)))

    print(f"Beta估计设置:")
    print(f"  窗口期: {window_months} 个月")
    print(f"  最少需要: {min_periods} 个月有数据")
    print("="*60)

    betas = []
    months = []
    Stkcds = []
    data_counts = []

    grouped = data.groupby('Stkcd')

    for Stkcd, group in grouped:
        group = group.set_index('month').sort_index()
        end_months = group.index.unique()

        for end_month in end_months:
            start_month = end_month - pd.DateOffset(months=window_months)
            window_data = group.loc[start_month:end_month]

            if len(window_data) >= min_periods:
                window_data_clean = window_data.dropna(subset=['ret', 'MKT'])

                if len(window_data_clean) >= min_periods:
                    X = sm.add_constant(window_data_clean['MKT'])
                    y = window_data_clean['ret']
                    model = sm.OLS(y, X).fit()

                    beta = model.params['MKT']
                    betas.append(beta)
                    months.append(end_month)
                    Stkcds.append(Stkcd)
                    data_counts.append(len(window_data_clean))

    result_df = pd.DataFrame({'Stkcd': Stkcds, 'month': months, 'beta': betas, 'data_count': data_counts})

    print(f"\n估计完成:")
    print(f"  总观测数: {len(result_df)}")
    print(f"  平均数据点数: {result_df['data_count'].mean():.1f}")
    print(f"  数据点数统计:")
    print(result_df['data_count'].describe())

    return result_df

rolling_betas = rolling_beta_per_stock(cross)

print(rolling_betas)

Beta估计设置:
  窗口期: 60 个月
  最少需要: 40 个月有数据

估计完成:
  总观测数: 630276
  平均数据点数: 58.1
  数据点数统计:
count    630276.000000
mean         58.100191
std           5.075211
min          40.000000
25%          57.000000
50%          61.000000
75%          61.000000
max          61.000000
Name: data_count, dtype: float64
         Stkcd      month      beta  data_count
0       000001 1998-04-30  1.155400          40
1       000001 1998-05-31  1.148875          41
2       000001 1998-06-30  1.167744          42
3       000001 1998-07-31  1.167735          43
4       000001 1998-08-31  1.152495          44
...        ...        ...       ...         ...
630271  920964 2024-08-31  0.725409          49
630272  920964 2024-09-30  0.983053          50
630273  920964 2024-10-31  0.877199          51
630274  920964 2024-11-30  0.860914          52
630275  920964 2024-12-31  0.872943          53

[630276 rows x 4 columns]


In [23]:
rolling_betas.to_csv('/Users/liuwanting/Desktop/pythonhomework/rolling_betas.csv', index=False)

In [24]:
rolling_betas

,Stkcd,month,beta,data_count
0,000001,1998-04-30,1.155400,40
1,000001,1998-05-31,1.148875,41
2,000001,1998-06-30,1.167744,42
3,000001,1998-07-31,1.167735,43
4,000001,1998-08-31,1.152495,44
...,...,...,...,...
630271,920964,2024-08-31,0.725409,49
630272,920964,2024-09-30,0.983053,50
630273,920964,2024-10-31,0.877199,51
630274,920964,2024-11-30,0.860914,52


In [25]:
cross_beta = pd.merge(cross,rolling_betas,on=("Stkcd",'month'),how='left')
cross_beta = cross_beta.dropna(subset=['beta'])
cross_beta

,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,floatingvalue,totalvalue,Ndaytrd,Return,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M,month,rfmonth,list_month,ret,next_ret,Cumsum_tradingday,MKT,beta,data_count
39,000001,1998-04,1.0,21.50,30.0,20.86,135781919.0,2.931748e+09,2.233903e+10,3.237153e+10,21.0,-0.032916,-0.032916,4.0,1997-08-25,NaN,NaN,1998-04-30,0.004249,85.0,-0.037165,-0.042121,235.0,0.085311,1.155400,40.0
40,000001,1998-05,4.0,20.90,29.0,20.07,108329870.0,2.208166e+09,2.149301e+10,3.114557e+10,20.0,-0.037872,-0.037872,4.0,1997-08-25,NaN,NaN,1998-05-31,0.004249,86.0,-0.042121,-0.151235,235.0,0.050219,1.148875,41.0
41,000001,1998-06,1.0,20.10,30.0,17.12,91965412.0,1.746011e+09,1.833385e+10,2.656762e+10,22.0,-0.146986,-0.146986,4.0,1997-08-25,NaN,NaN,1998-06-30,0.004249,87.0,-0.151235,0.004287,242.0,-0.066011,1.167744,42.0
42,000001,1998-07,1.0,17.58,31.0,17.26,102722557.0,1.740172e+09,1.848378e+10,2.678488e+10,23.0,0.008178,0.008178,4.0,1997-08-25,NaN,NaN,1998-07-31,0.003891,88.0,0.004287,-0.068781,244.0,-0.016875,1.167735,43.0
43,000001,1998-08,3.0,17.28,31.0,16.14,71852835.0,1.172085e+09,1.728437e+10,2.504681e+10,21.0,-0.064890,-0.064890,4.0,1997-08-25,NaN,NaN,1998-08-31,0.003891,89.0,-0.068781,-0.026815,244.0,-0.125304,1.152495,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109595,920964,2024-08,1.0,3.03,30.0,2.77,36090137.0,1.037190e+08,4.969253e+08,7.235462e+08,22.0,-0.082782,-0.082781,64.0,2023-11-21,0.0,0.0,2024-08-31,0.001118,50.0,-0.083900,0.305740,242.0,-0.032586,0.725409,49.0
2109596,920964,2024-09,2.0,2.80,30.0,3.62,49005108.0,1.488490e+08,6.494114e+08,9.455730e+08,19.0,0.306858,0.306859,64.0,2023-11-21,0.0,0.0,2024-09-30,0.001118,51.0,0.305740,2.399433,241.0,0.194874,0.983053,50.0
2109597,920964,2024-10,8.0,4.36,31.0,12.31,219750340.0,1.478570e+09,2.208358e+09,3.215470e+09,14.0,2.400551,2.400552,64.0,2023-11-21,0.0,0.0,2024-10-31,0.001118,52.0,2.399433,-0.218827,238.0,-0.004527,0.877199,51.0
2109598,920964,2024-11,1.0,13.40,29.0,9.63,628989566.0,7.034681e+09,1.727578e+09,2.515433e+09,21.0,-0.217709,-0.217709,64.0,2023-11-21,0.0,0.0,2024-11-30,0.001118,53.0,-0.218827,-0.272146,237.0,0.008606,0.860914,52.0


In [26]:
cross_beta = cross_beta[cross_beta['Ndaytrd'] >= 7]
cross_beta = cross_beta[cross_beta['Clsdt'] >= 5]
cross_beta = cross_beta[cross_beta['list_month'] > 6]
cross_beta = cross_beta[cross_beta['Cumsum_tradingday'] >= 100]
cross_beta = cross_beta[(cross_beta['Markettype'] == 1) | (cross_beta['Markettype'] == 4) | (cross_beta['Markettype'] == 6)]
cross_beta

,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,floatingvalue,totalvalue,Ndaytrd,Return,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M,month,rfmonth,list_month,ret,next_ret,Cumsum_tradingday,MKT,beta,data_count
39,000001,1998-04,1.0,21.50,30.0,20.86,135781919.0,2.931748e+09,2.233903e+10,3.237153e+10,21.0,-0.032916,-0.032916,4.0,1997-08-25,NaN,NaN,1998-04-30,0.004249,85.0,-0.037165,-0.042121,235.0,0.085311,1.155400,40.0
40,000001,1998-05,4.0,20.90,29.0,20.07,108329870.0,2.208166e+09,2.149301e+10,3.114557e+10,20.0,-0.037872,-0.037872,4.0,1997-08-25,NaN,NaN,1998-05-31,0.004249,86.0,-0.042121,-0.151235,235.0,0.050219,1.148875,41.0
41,000001,1998-06,1.0,20.10,30.0,17.12,91965412.0,1.746011e+09,1.833385e+10,2.656762e+10,22.0,-0.146986,-0.146986,4.0,1997-08-25,NaN,NaN,1998-06-30,0.004249,87.0,-0.151235,0.004287,242.0,-0.066011,1.167744,42.0
42,000001,1998-07,1.0,17.58,31.0,17.26,102722557.0,1.740172e+09,1.848378e+10,2.678488e+10,23.0,0.008178,0.008178,4.0,1997-08-25,NaN,NaN,1998-07-31,0.003891,88.0,0.004287,-0.068781,244.0,-0.016875,1.167735,43.0
43,000001,1998-08,3.0,17.28,31.0,16.14,71852835.0,1.172085e+09,1.728437e+10,2.504681e+10,21.0,-0.064890,-0.064890,4.0,1997-08-25,NaN,NaN,1998-08-31,0.003891,89.0,-0.068781,-0.026815,244.0,-0.125304,1.152495,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776239,605507,2024-12,2.0,22.07,31.0,20.83,61892279.0,1.307565e+09,6.284873e+09,1.164029e+10,22.0,-0.047989,-0.047989,1.0,2023-02-02,0.0,0.0,2024-12-31,0.001118,41.0,-0.049107,NaN,242.0,-0.003797,0.604456,40.0
1777679,605577,2024-12,2.0,16.93,31.0,14.53,228878763.0,3.862518e+09,6.457778e+09,6.457778e+09,22.0,-0.135119,-0.135119,1.0,2024-09-30,0.0,0.0,2024-12-31,0.001118,41.0,-0.136237,NaN,242.0,-0.003797,0.764475,40.0
1778039,605580,2024-12,2.0,9.88,31.0,11.08,154703585.0,1.623849e+09,3.102400e+09,3.102400e+09,22.0,0.132924,0.132924,1.0,2024-08-26,0.0,0.0,2024-12-31,0.001118,41.0,0.131806,NaN,242.0,-0.003797,0.785369,40.0
1778399,605588,2024-12,2.0,47.96,31.0,40.76,41273627.0,1.873835e+09,2.979538e+09,2.995151e+09,22.0,-0.150302,-0.150302,1.0,2024-12-17,0.0,0.0,2024-12-31,0.001118,41.0,-0.151420,NaN,242.0,-0.003797,0.976478,40.0


In [27]:
fenweishu = pd.DataFrame(
    cross_beta.groupby(['month'])['beta'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))
fenweishu = fenweishu.reset_index()
fenweishu = fenweishu.pivot_table(index='month',columns='level_1',values='beta')
fenweishu.columns = ['one','two','three','four','five','six','seven','eight','nine']
fenweishu

,one,two,three,four,five,six,seven,eight,nine
month,,,,,,,,,
1998-04-30,0.707349,0.781328,0.847070,0.918619,0.986271,1.053890,1.147653,1.263057,1.403121
1998-05-31,0.710665,0.786197,0.852553,0.921351,0.992781,1.057841,1.143836,1.276886,1.410021
1998-06-30,0.720040,0.782919,0.868701,0.920163,0.991961,1.060550,1.148936,1.277518,1.404875
1998-07-31,0.708720,0.780527,0.862017,0.919622,0.994468,1.065228,1.141907,1.270604,1.398097
1998-08-31,0.708167,0.774056,0.849154,0.922411,0.989177,1.068256,1.151609,1.247518,1.387920
...,...,...,...,...,...,...,...,...,...
2024-08-31,0.527212,0.690668,0.808727,0.903639,1.005142,1.103606,1.236126,1.375513,1.610912
2024-09-30,0.613942,0.747197,0.844887,0.929128,1.020618,1.112952,1.212442,1.330217,1.528623
2024-10-31,0.609603,0.741257,0.840805,0.923708,1.015869,1.106596,1.205720,1.327996,1.526917


In [28]:
portfolio = pd.merge(cross_beta,fenweishu,on='month')
portfolio

,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,floatingvalue,totalvalue,Ndaytrd,Return,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M,month,rfmonth,list_month,ret,next_ret,Cumsum_tradingday,MKT,beta,data_count,one,two,three,four,five,six,seven,eight,nine
0,000001,1998-04,1.0,21.50,30.0,20.86,135781919.0,2.931748e+09,2.233903e+10,3.237153e+10,21.0,-0.032916,-0.032916,4.0,1997-08-25,NaN,NaN,1998-04-30,0.004249,85.0,-0.037165,-0.042121,235.0,0.085311,1.155400,40.0,0.707349,0.781328,0.847070,0.918619,0.986271,1.053890,1.147653,1.263057,1.403121
1,000001,1998-05,4.0,20.90,29.0,20.07,108329870.0,2.208166e+09,2.149301e+10,3.114557e+10,20.0,-0.037872,-0.037872,4.0,1997-08-25,NaN,NaN,1998-05-31,0.004249,86.0,-0.042121,-0.151235,235.0,0.050219,1.148875,41.0,0.710665,0.786197,0.852553,0.921351,0.992781,1.057841,1.143836,1.276886,1.410021
2,000001,1998-06,1.0,20.10,30.0,17.12,91965412.0,1.746011e+09,1.833385e+10,2.656762e+10,22.0,-0.146986,-0.146986,4.0,1997-08-25,NaN,NaN,1998-06-30,0.004249,87.0,-0.151235,0.004287,242.0,-0.066011,1.167744,42.0,0.720040,0.782919,0.868701,0.920163,0.991961,1.060550,1.148936,1.277518,1.404875
3,000001,1998-07,1.0,17.58,31.0,17.26,102722557.0,1.740172e+09,1.848378e+10,2.678488e+10,23.0,0.008178,0.008178,4.0,1997-08-25,NaN,NaN,1998-07-31,0.003891,88.0,0.004287,-0.068781,244.0,-0.016875,1.167735,43.0,0.708720,0.780527,0.862017,0.919622,0.994468,1.065228,1.141907,1.270604,1.398097
4,000001,1998-08,3.0,17.28,31.0,16.14,71852835.0,1.172085e+09,1.728437e+10,2.504681e+10,21.0,-0.064890,-0.064890,4.0,1997-08-25,NaN,NaN,1998-08-31,0.003891,89.0,-0.068781,-0.026815,244.0,-0.125304,1.152495,44.0,0.708167,0.774056,0.849154,0.922411,0.989177,1.068256,1.151609,1.247518,1.387920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502361,605507,2024-12,2.0,22.07,31.0,20.83,61892279.0,1.307565e+09,6.284873e+09,1.164029e+10,22.0,-0.047989,-0.047989,1.0,2023-02-02,0.0,0.0,2024-12-31,0.001118,41.0,-0.049107,NaN,242.0,-0.003797,0.604456,40.0,0.609035,0.742393,0.841648,0.923380,1.017028,1.108646,1.208609,1.329028,1.529482
502362,605577,2024-12,2.0,16.93,31.0,14.53,228878763.0,3.862518e+09,6.457778e+09,6.457778e+09,22.0,-0.135119,-0.135119,1.0,2024-09-30,0.0,0.0,2024-12-31,0.001118,41.0,-0.136237,NaN,242.0,-0.003797,0.764475,40.0,0.609035,0.742393,0.841648,0.923380,1.017028,1.108646,1.208609,1.329028,1.529482
502363,605580,2024-12,2.0,9.88,31.0,11.08,154703585.0,1.623849e+09,3.102400e+09,3.102400e+09,22.0,0.132924,0.132924,1.0,2024-08-26,0.0,0.0,2024-12-31,0.001118,41.0,0.131806,NaN,242.0,-0.003797,0.785369,40.0,0.609035,0.742393,0.841648,0.923380,1.017028,1.108646,1.208609,1.329028,1.529482
502364,605588,2024-12,2.0,47.96,31.0,40.76,41273627.0,1.873835e+09,2.979538e+09,2.995151e+09,22.0,-0.150302,-0.150302,1.0,2024-12-17,0.0,0.0,2024-12-31,0.001118,41.0,-0.151420,NaN,242.0,-0.003797,0.976478,40.0,0.609035,0.742393,0.841648,0.923380,1.017028,1.108646,1.208609,1.329028,1.529482


In [29]:
portfolio['sort'] = np.where(
    portfolio['beta'] <= portfolio['one'], 'P1',
    np.where(
        portfolio['beta'] <= portfolio['two'], 'P2',
        np.where(
            portfolio['beta'] <= portfolio['three'], 'P3',
            np.where(
                portfolio['beta'] <= portfolio['four'], 'P4',
                np.where(
                    portfolio['beta'] <= portfolio['five'], 'P5',
                    np.where(
                        portfolio['beta'] <= portfolio['six'], 'P6',
                        np.where(
                            portfolio['beta'] <= portfolio['seven'], 'P7',
                            np.where(
                                portfolio['beta'] <= portfolio['eight'], 'P8',
                                np.where(
                                    portfolio['beta'] <= portfolio['nine'],
                                    'P9', 'Pmax')))))))))
portfolio = portfolio.dropna(subset=['floatingvalue','next_ret','beta'])
portfolio

,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,floatingvalue,totalvalue,Ndaytrd,Return,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M,month,rfmonth,list_month,ret,next_ret,Cumsum_tradingday,MKT,beta,data_count,one,two,three,four,five,six,seven,eight,nine,sort
0,000001,1998-04,1.0,21.50,30.0,20.86,135781919.0,2.931748e+09,2.233903e+10,3.237153e+10,21.0,-0.032916,-0.032916,4.0,1997-08-25,NaN,NaN,1998-04-30,0.004249,85.0,-0.037165,-0.042121,235.0,0.085311,1.155400,40.0,0.707349,0.781328,0.847070,0.918619,0.986271,1.053890,1.147653,1.263057,1.403121,P8
1,000001,1998-05,4.0,20.90,29.0,20.07,108329870.0,2.208166e+09,2.149301e+10,3.114557e+10,20.0,-0.037872,-0.037872,4.0,1997-08-25,NaN,NaN,1998-05-31,0.004249,86.0,-0.042121,-0.151235,235.0,0.050219,1.148875,41.0,0.710665,0.786197,0.852553,0.921351,0.992781,1.057841,1.143836,1.276886,1.410021,P8
2,000001,1998-06,1.0,20.10,30.0,17.12,91965412.0,1.746011e+09,1.833385e+10,2.656762e+10,22.0,-0.146986,-0.146986,4.0,1997-08-25,NaN,NaN,1998-06-30,0.004249,87.0,-0.151235,0.004287,242.0,-0.066011,1.167744,42.0,0.720040,0.782919,0.868701,0.920163,0.991961,1.060550,1.148936,1.277518,1.404875,P8
3,000001,1998-07,1.0,17.58,31.0,17.26,102722557.0,1.740172e+09,1.848378e+10,2.678488e+10,23.0,0.008178,0.008178,4.0,1997-08-25,NaN,NaN,1998-07-31,0.003891,88.0,0.004287,-0.068781,244.0,-0.016875,1.167735,43.0,0.708720,0.780527,0.862017,0.919622,0.994468,1.065228,1.141907,1.270604,1.398097,P8
4,000001,1998-08,3.0,17.28,31.0,16.14,71852835.0,1.172085e+09,1.728437e+10,2.504681e+10,21.0,-0.064890,-0.064890,4.0,1997-08-25,NaN,NaN,1998-08-31,0.003891,89.0,-0.068781,-0.026815,244.0,-0.125304,1.152495,44.0,0.708167,0.774056,0.849154,0.922411,0.989177,1.068256,1.151609,1.247518,1.387920,P8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502355,605500,2024-07,1.0,6.16,31.0,6.40,32852013.0,2.035688e+08,2.652160e+09,2.652160e+09,23.0,0.067517,0.065041,1.0,2024-06-24,0.0,0.0,2024-07-31,0.001184,44.0,0.066333,-0.035493,243.0,-0.000624,0.796606,43.0,0.523361,0.692383,0.806922,0.904072,1.002450,1.105938,1.233599,1.373471,1.608908,P3
502356,605500,2024-08,1.0,6.40,30.0,6.18,24283030.0,1.500448e+08,2.560992e+09,2.560992e+09,22.0,-0.034375,-0.034375,1.0,2024-06-24,0.0,0.0,2024-08-31,0.001118,45.0,-0.035493,0.152604,242.0,-0.032586,0.798576,44.0,0.527212,0.690668,0.808727,0.903639,1.005142,1.103606,1.236126,1.375513,1.610912,P3
502357,605500,2024-09,2.0,6.17,30.0,7.13,27875613.0,1.777782e+08,2.954672e+09,2.954672e+09,19.0,0.153722,0.153722,1.0,2024-06-24,0.0,0.0,2024-09-30,0.001118,46.0,0.152604,-0.012338,241.0,0.194874,0.800304,45.0,0.613942,0.747197,0.844887,0.929128,1.020618,1.112952,1.212442,1.330217,1.528623,P3
502358,605500,2024-10,8.0,7.83,31.0,7.05,49883893.0,3.539763e+08,2.921520e+09,2.921520e+09,18.0,-0.011220,-0.011220,1.0,2024-06-24,0.0,0.0,2024-10-31,0.001118,47.0,-0.012338,0.137889,242.0,-0.004527,0.800487,46.0,0.609603,0.741257,0.840805,0.923708,1.015869,1.106596,1.205720,1.327996,1.526917,P3


In [30]:
portfolio = portfolio.dropna(subset=['next_ret','floatingvalue','beta'])
portfolio_beta =  pd.DataFrame(
    portfolio.groupby(['month','sort']).apply(lambda x: np.average(x['next_ret'],weights = x['floatingvalue']),include_groups=False))
portfolio_beta

0
month      sort          
1998-04-30 P1    0.073639
           P2    0.073060
           P3    0.059682
           P4    0.126568
           P5    0.060105
...                   ...
2024-11-30 P6   -0.002935
           P7   -0.022196
           P8   -0.021325
           P9   -0.036926
           Pmax -0.050939

[3200 rows x 1 columns]

In [31]:
portfolio_beta = portfolio_beta.reset_index()
portfolio_beta.columns = ['month', 'sort', 'p']
portfolio_beta['month'] = portfolio_beta['month'] + MonthEnd(1)
portfolio_beta

,month,sort,p
0,1998-05-31,P1,0.073639
1,1998-05-31,P2,0.073060
2,1998-05-31,P3,0.059682
3,1998-05-31,P4,0.126568
4,1998-05-31,P5,0.060105
...,...,...,...
3195,2024-12-31,P6,-0.002935
3196,2024-12-31,P7,-0.022196
3197,2024-12-31,P8,-0.021325
3198,2024-12-31,P9,-0.036926


In [32]:
portfolio_beta = portfolio_beta.pivot_table(index='month',
                                            columns='sort',
                                            values='p')
portfolio_beta['My_portfolio'] = portfolio_beta['Pmax'] - portfolio_beta['P1']
portfolio_beta

sort,P1,P2,P3,P4,P5,P6,P7,P8,P9,Pmax,My_portfolio
month,,,,,,,,,,,
1998-05-31,0.073639,0.073060,0.059682,0.126568,0.060105,0.110886,0.070398,0.014426,0.066955,0.065713,-0.007925
1998-06-30,-0.068016,-0.083137,-0.033254,-0.073349,-0.042640,-0.083524,-0.059467,-0.133064,-0.125269,-0.091285,-0.023269
1998-07-31,0.022387,0.002304,-0.010380,-0.030658,-0.022810,-0.021796,-0.021273,-0.005191,0.017707,0.013602,-0.008785
1998-08-31,-0.104332,-0.088575,-0.109517,-0.094662,-0.144915,-0.150273,-0.133799,-0.111460,-0.133402,-0.137561,-0.033229
1998-09-30,0.062278,0.071437,0.070417,0.035619,0.073661,0.034641,0.107009,0.023006,0.067436,0.085260,0.022982
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.009478,-0.040298,-0.038603,-0.042049,-0.051192,-0.015255,-0.039640,-0.023992,-0.043447,-0.056096,-0.046618
2024-09-30,0.083155,0.182333,0.161097,0.225707,0.206607,0.217508,0.247355,0.228438,0.225740,0.238674,0.155519
2024-10-31,-0.052130,-0.031471,-0.012259,0.032790,-0.008649,-0.028356,0.007237,0.005666,-0.009967,0.042866,0.094996


In [33]:
portfolio_beta = portfolio_beta['1995-01':'2024-12']
portfolio_beta

sort,P1,P2,P3,P4,P5,P6,P7,P8,P9,Pmax,My_portfolio
month,,,,,,,,,,,
1998-05-31,0.073639,0.073060,0.059682,0.126568,0.060105,0.110886,0.070398,0.014426,0.066955,0.065713,-0.007925
1998-06-30,-0.068016,-0.083137,-0.033254,-0.073349,-0.042640,-0.083524,-0.059467,-0.133064,-0.125269,-0.091285,-0.023269
1998-07-31,0.022387,0.002304,-0.010380,-0.030658,-0.022810,-0.021796,-0.021273,-0.005191,0.017707,0.013602,-0.008785
1998-08-31,-0.104332,-0.088575,-0.109517,-0.094662,-0.144915,-0.150273,-0.133799,-0.111460,-0.133402,-0.137561,-0.033229
1998-09-30,0.062278,0.071437,0.070417,0.035619,0.073661,0.034641,0.107009,0.023006,0.067436,0.085260,0.022982
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.009478,-0.040298,-0.038603,-0.042049,-0.051192,-0.015255,-0.039640,-0.023992,-0.043447,-0.056096,-0.046618
2024-09-30,0.083155,0.182333,0.161097,0.225707,0.206607,0.217508,0.247355,0.228438,0.225740,0.238674,0.155519
2024-10-31,-0.052130,-0.031471,-0.012259,0.032790,-0.008649,-0.028356,0.007237,0.005666,-0.009967,0.042866,0.094996


In [34]:
model = smf.ols('My_portfolio ~ 1',
                 data=portfolio_beta['1995-01':'2024-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           My_portfolio   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 16 Nov 2025   Prob (F-statistic):                nan
Time:                        15:39:47   Log-Likelihood:                 447.13
No. Observations:                 320   AIC:                            -892.3
Df Residuals:                     319   BIC:                            -888.5
Df Model:                           0                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0062      0.003     -2.249      0.0

### 是否存在α

In [35]:
cross_beta2 = cross_beta.copy()
cross_beta2 = cross_beta2.set_index(['Stkcd', 'month'])
cross_beta2 = cross_beta2.dropna(subset=['next_ret','beta'])
cross_beta2 = cross_beta2[(cross_beta2.index.get_level_values('month') >= '2000-01-31') &
                          (cross_beta2.index.get_level_values('month') <= '2024-12-31')]
cross_beta2

Trdmnt  Opndt  Mopnprc  Clsdt  Mclsprc     Mnshrtrd  \
Stkcd  month                                                              
000001 2000-01-31  2000-01    4.0    17.50   28.0    18.53  178385288.0   
       2000-02-29  2000-02   14.0    19.20   29.0    18.32  220816880.0   
       2000-03-31  2000-03    1.0    18.40   31.0    18.37  362256029.0   
       2000-04-30  2000-04    3.0    18.38   28.0    19.05  208567616.0   
       2000-05-31  2000-05    8.0    19.12   31.0    18.00   98004336.0   
...                    ...    ...      ...    ...      ...          ...   
605500 2024-07-31  2024-07    1.0     6.16   31.0     6.40   32852013.0   
       2024-08-31  2024-08    1.0     6.40   30.0     6.18   24283030.0   
       2024-09-30  2024-09    2.0     6.17   30.0     7.13   27875613.0   
       2024-10-31  2024-10    8.0     7.83   31.0     7.05   49883893.0   
       2024-11-30  2024-11    1.0     7.06   29.0     8.03   66866137.0   

                       Mnvaltrd  floatingvalue    totalvalue  Ndaytrd  \
Stkcd  month                                                            
000001 2000-01-31  3.299056e+09   1.984382e+10  2.875573e+10     19.0   
       2000-02-29  4.175318e+09   1.961893e+10  2.842984e+10     12.0   
       2000-03-31  6.859299e+09   1.967248e+10  2.850743e+10     23.0   
       2000-04-30  3.978433e+09   2.040069e+10  2.956269e+10     20.0   
       2000-05-31  1.779608e+09   1.927624e+10  2.793325e+10     18.0   
...                         ...            ...           ...      ...   
605500 2024-07-31  2.035688e+08   2.652160e+09  2.652160e+09     23.0   
       2024-08-31  1.500448e+08   2.560992e+09  2.560992e+09     22.0   
       2024-09-30  1.777782e+08   2.954672e+09  2.954672e+09     19.0   
       2024-10-31  3.539763e+08   2.921520e+09  2.921520e+09     18.0   
       2024-11-30  5.037812e+08   3.327632e+09  3.327632e+09     21.0   

                     Return    Mretnd  Markettype    Capchgdt  Ahshrtrd_M  \
Stkcd  month                                                                
000001 2000-01-31  0.061891  0.061891         4.0  1997-08-25         NaN   
       2000-02-29 -0.011333 -0.011333         4.0  1997-08-25         NaN   
       2000-03-31  0.002729  0.002729         4.0  1997-08-25         NaN   
       2000-04-30  0.037017  0.037017         4.0  1997-08-25         NaN   
       2000-05-31 -0.055118 -0.055118         4.0  1997-08-25         NaN   
...                     ...       ...         ...         ...         ...   
605500 2024-07-31  0.067517  0.065041         1.0  2024-06-24         0.0   
       2024-08-31 -0.034375 -0.034375         1.0  2024-06-24         0.0   
       2024-09-30  0.153722  0.153722         1.0  2024-06-24         0.0   
       2024-10-31 -0.011220 -0.011220         1.0  2024-06-24         0.0   
       2024-11-30  0.139007  0.139007         1.0  2024-06-24         0.0   

                   Ahvaltrd_M   rfmonth  list_month       ret  next_ret  \
Stkcd  month                                                              
000001 2000-01-31         NaN  0.001856       106.0  0.060035 -0.013189   
       2000-02-29         NaN  0.001856       107.0 -0.013189  0.000873   
       2000-03-31         NaN  0.001856       108.0  0.000873  0.035161   
       2000-04-30         NaN  0.001856       109.0  0.035161 -0.056974   
       2000-05-31         NaN  0.001856       110.0 -0.056974  0.005366   
...                       ...       ...         ...       ...       ...   
605500 2024-07-31         0.0  0.001184        44.0  0.066333 -0.035493   
       2024-08-31         0.0  0.001118        45.0 -0.035493  0.152604   
       2024-09-30         0.0  0.001118        46.0  0.152604 -0.012338   
       2024-10-31         0.0  0.001118        47.0 -0.012338  0.137889   
       2024-11-30         0.0  0.001118        48.0  0.137889 -0.134368   

                   Cumsum_tradingday       MKT      beta  data_count  
Stkcd  month                                 

In [37]:
from linearmodels import FamaMacBeth
model = FamaMacBeth.from_formula('next_ret ~ 1 + beta', data=cross_beta2.dropna(subset=['next_ret','beta']))
res = model.fit(cov_type= 'kernel',debiased = False,bandwidth=6)
print(res.summary)

                            FamaMacBeth Estimation Summary                           
Dep. Variable:                    next_ret   R-squared:                     1.449e-05
Estimator:                     FamaMacBeth   R-squared (Between):             -0.0172
No. Observations:                   490287   R-squared (Within):               0.0001
Date:                     Sun, Nov 16 2025   R-squared (Overall):           1.449e-05
Time:                             15:52:39   Log-likelihood                 2.758e+05
Cov. Estimator:    Fama-MacBeth Kernel Cov                                           
                                             F-statistic:                      7.1056
Entities:                             3236   P-value                           0.0077
Avg Obs:                            151.51   Distribution:                F(1,490285)
Min Obs:                            1.0000                                           
Max Obs:                            299.00   F-statist

### My Code for Fama MacBeth

In [38]:
import statsmodels.api as sm

def Fama_MacBeth(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

coef = cross_beta2.dropna(subset=['next_ret','beta']).groupby('month').apply(Fama_MacBeth, 'next_ret', ['beta'])
coef

,beta,intercept
month,,
2000-01-31,0.025960,0.124545
2000-02-29,-0.067173,0.168086
2000-03-31,-0.023504,0.041805
2000-04-30,-0.007965,0.044455
2000-05-31,-0.022458,0.044424
...,...,...
2024-07-31,-0.011379,-0.021878
2024-08-31,0.023007,0.184893
2024-09-30,0.041929,0.020726


In [39]:
model_alpha = smf.ols('intercept ~ 1',
                 data=coef['2000-01':'2024-11']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_alpha.summary())

                            OLS Regression Results                            
Dep. Variable:              intercept   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 16 Nov 2025   Prob (F-statistic):                nan
Time:                        15:54:01   Log-Likelihood:                 341.82
No. Observations:                 299   AIC:                            -681.6
Df Residuals:                     298   BIC:                            -677.9
Df Model:                           0                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0137      0.006      2.346      0.0

In [40]:
model_beta = smf.ols('beta ~ 1',
                 data=coef['1994-12':'2024-11']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_beta.summary())

                            OLS Regression Results                            
Dep. Variable:                   beta   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 16 Nov 2025   Prob (F-statistic):                nan
Time:                        15:54:05   Log-Likelihood:                 593.30
No. Observations:                 299   AIC:                            -1185.
Df Residuals:                     298   BIC:                            -1181.
Df Model:                           0                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0035      0.002     -2.066      0.0